<a href="https://colab.research.google.com/github/phdsasarun/Learn-Github/blob/main/Modi_Modi_CKD_0505.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# การวิเคราะห์แนวโน้มการเกิดโรคไตเรื้อรัง (CKD) จากข้อมูลเวชระเบียนอิเล็กทรอนิกส์
# ด้วยเทคนิคการเรียนรู้ของเครื่อง (Machine Learning)

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')

# Set Thai font for matplotlib
plt.rcParams['font.family'] = ['DejaVu Sans']
plt.style.use('default')

print("=" * 80)
print("การวิเคราะห์แนวโน้มการเกิดโรคไตเรื้อรัง (CKD)")
print("จากข้อมูลเวชระเบียนอิเล็กทรอนิกส์ด้วยเทคนิคการเรียนรู้ของเครื่อง")
print("=" * 80)

# 1. โหลดและตรวจสอบข้อมูล
print("\n1. การโหลดและตรวจสอบข้อมูل")
print("-" * 50)

# ในการใช้งานจริง ให้ดาวน์โหลดไฟล์จาก Google Drive และอัพโหลดไปยัง Colab
# หรือใช้คำสั่งด้านล่างเพื่อโหลดโดยตรง:
# !gdown --id 12tU2nSdEy7OfbZtHr8pxUuuh00KxWkod

# สร้างข้อมูลตัวอย่างสำหรับการทดสอบ (ในการใช้งานจริงให้แทนที่ด้วยข้อมูลจริง)
np.random.seed(42)
n_samples = 1000

# สร้างข้อมูลตัวอย่างตามลักษณะข้อมูลทางการแพทย์
data = {
    'Age': np.random.normal(55, 15, n_samples).astype(int),
    'Blood_Pressure': np.random.normal(130, 20, n_samples),
    'Albumin_Urine': np.random.exponential(2, n_samples),
    'Random_Blood_Glucose': np.random.normal(150, 50, n_samples),
    'Serum_Creatinine': np.random.exponential(1.2, n_samples),
    'Hemoglobin': np.random.normal(12, 2, n_samples),
    'eGFR': np.random.normal(80, 30, n_samples),
    'Cholesterol': np.random.normal(200, 40, n_samples),
    'BMI': np.random.normal(25, 5, n_samples),
    'Diabetes_Duration': np.random.exponential(5, n_samples)
}

df = pd.DataFrame(data)

# สร้าง Target variable ตามเกณฑ์ทางการแพทย์
def classify_ckd_stage(row):
    egfr = row['eGFR']
    creatinine = row['Serum_Creatinine']
    albumin = row['Albumin_Urine']

    if egfr >= 90 and creatinine <= 1.2 and albumin <= 0.3:
        return 1  # No Disease
    elif egfr >= 60 and creatinine <= 1.5:
        return 2  # Low Risk
    elif egfr >= 30 and creatinine <= 3.0:
        return 3  # Moderate Risk
    elif egfr >= 15:
        return 4  # High Risk
    else:
        return 5  # Severe Disease

df['Target'] = df.apply(classify_ckd_stage, axis=1)

# แก้ไขค่าที่ไม่สมเหตุสมผล
df['Age'] = np.clip(df['Age'], 18, 100)
df['Blood_Pressure'] = np.clip(df['Blood_Pressure'], 80, 200)
df['Random_Blood_Glucose'] = np.clip(df['Random_Blood_Glucose'], 70, 400)
df['Hemoglobin'] = np.clip(df['Hemoglobin'], 6, 18)
df['eGFR'] = np.clip(df['eGFR'], 5, 150)
df['Cholesterol'] = np.clip(df['Cholesterol'], 120, 350)
df['BMI'] = np.clip(df['BMI'], 15, 45)

print("ข้อมูลพื้นฐาน:")
print(f"จำนวนตัวอย่าง: {len(df)}")
print(f"จำนวนฟีเจอร์: {len(df.columns)-1}")
print("\nตัวอย่างข้อมูล 5 แถวแรก:")
print(df.head())

print("\nข้อมูลสถิติพื้นฐาน:")
print(df.describe())

# 2. การกระจายของ Target Variable
print("\n2. การกระจายของระดับโรคไตเรื้อรัง")
print("-" * 50)

target_counts = df['Target'].value_counts().sort_index()
target_labels = {
    1: 'No Disease (ไม่มีภาวะโรคไต)',
    2: 'Low Risk (ความเสี่ยงต่ำ)',
    3: 'Moderate Risk (ความเสี่ยงปานกลาง)',
    4: 'High Risk (ความเสี่ยงสูง)',
    5: 'Severe Disease (โรคไตเรื้อรังร้ายแรง)'
}

print("การกระจายของระดับโรค:")
for stage, count in target_counts.items():
    percentage = (count / len(df)) * 100
    print(f"ระดับ {stage}: {target_labels[stage]} = {count} ราย ({percentage:.1f}%)")

# 3. เกณฑ์การวินิจฉัยและตัวชี้วัดสำคัญ
print("\n3. เกณฑ์การวินิจฉัยและตัวชี้วัดสำคัญ")
print("-" * 50)

diagnostic_criteria = {
    'Age (ปี)': {
        'Normal': '< 65',
        'Risk Factor': '≥ 65'
    },
    'Blood Pressure (mmHg)': {
        'Normal': '< 130/80',
        'High': '≥ 140/90',
        'Severe': '≥ 180/110'
    },
    'Albumin in Urine (mg/dL)': {
        'Normal': '< 0.3',
        'Microalbuminuria': '0.3-2.9',
        'Macroalbuminuria': '≥ 3.0'
    },
    'Random Blood Glucose (mg/dL)': {
        'Normal': '< 140',
        'Prediabetes': '140-199',
        'Diabetes': '≥ 200'
    },
    'Serum Creatinine (mg/dL)': {
        'Normal (Male)': '0.7-1.3',
        'Normal (Female)': '0.6-1.1',
        'Elevated': '> 1.5'
    },
    'Hemoglobin (g/dL)': {
        'Normal (Male)': '13.8-17.2',
        'Normal (Female)': '12.1-15.1',
        'Anemia': '< 12'
    },
    'eGFR (mL/min/1.73m²)': {
        'Normal': '≥ 90',
        'Mild decrease': '60-89',
        'Moderate decrease': '30-59',
        'Severe decrease': '15-29',
        'Kidney failure': '< 15'
    },
    'BMI (kg/m²)': {
        'Underweight': '< 18.5',
        'Normal': '18.5-24.9',
        'Overweight': '25-29.9',
        'Obese': '≥ 30'
    }
}

for criterion, ranges in diagnostic_criteria.items():
    print(f"\n{criterion}:")
    for category, range_val in ranges.items():
        print(f"  - {category}: {range_val}")

# 4. การเตรียมข้อมูล
print("\n4. การเตรียมข้อมูลสำหรับการวิเคราะห์")
print("-" * 50)

# แยกฟีเจอร์และ target
feature_columns = ['Age', 'Blood_Pressure', 'Albumin_Urine', 'Random_Blood_Glucose',
                  'Serum_Creatinine', 'Hemoglobin', 'eGFR', 'Cholesterol', 'BMI', 'Diabetes_Duration']
X = df[feature_columns]
y = df['Target']

# แบ่งข้อมูลเป็นชุดฝึกสอนและทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ปรับขนาดข้อมูล (Standardization)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"ขนาดข้อมูลฝึกสอน: {X_train.shape}")
print(f"ขนาดข้อมูลทดสอบ: {X_test.shape}")

# 5. การสร้างและฝึกสอนโมเดล
print("\n5. การสร้างและฝึกสอนโมเดล Machine Learning")
print("-" * 50)

# 5.1 Logistic Regression
print("\n5.1 Logistic Regression")
print("-" * 30)

lr_model = LogisticRegression(random_state=42, max_iter=1000, multi_class='ovr')
lr_model.fit(X_train_scaled, y_train)

# ทำนายผล
lr_pred = lr_model.predict(X_test_scaled)
lr_accuracy = accuracy_score(y_test, lr_pred)

print(f"Accuracy: {lr_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, lr_pred, target_names=[target_labels[i] for i in sorted(target_labels.keys())]))

# 5.2 Neural Network (Deep Learning)
print("\n5.2 Neural Network (Deep Learning)")
print("-" * 30)

nn_model = MLPClassifier(
    hidden_layer_sizes=(100, 50, 25),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1
)
nn_model.fit(X_train_scaled, y_train)

# ทำนายผล
nn_pred = nn_model.predict(X_test_scaled)
nn_accuracy = accuracy_score(y_test, nn_pred)

print(f"Accuracy: {nn_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, nn_pred, target_names=[target_labels[i] for i in sorted(target_labels.keys())]))

# 5.3 k-Fold Cross-Validation
print("\n5.3 k-Fold Cross-Validation")
print("-" * 30)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation สำหรับ Logistic Regression
lr_cv_scores = cross_val_score(lr_model, X_train_scaled, y_train, cv=kfold, scoring='accuracy')
print(f"Logistic Regression CV Scores: {lr_cv_scores}")
print(f"Logistic Regression CV Mean: {lr_cv_scores.mean():.4f} (+/- {lr_cv_scores.std() * 2:.4f})")

# Cross-validation สำหรับ Neural Network
nn_cv_scores = cross_val_score(nn_model, X_train_scaled, y_train, cv=kfold, scoring='accuracy')
print(f"Neural Network CV Scores: {nn_cv_scores}")
print(f"Neural Network CV Mean: {nn_cv_scores.mean():.4f} (+/- {nn_cv_scores.std() * 2:.4f})")

# 6. การเปรียบเทียบผลลัพธ์
print("\n6. การเปรียบเทียบผลลัพธ์")
print("-" * 50)

results_comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Neural Network'],
    'Test Accuracy': [lr_accuracy, nn_accuracy],
    'CV Mean Accuracy': [lr_cv_scores.mean(), nn_cv_scores.mean()],
    'CV Std': [lr_cv_scores.std(), nn_cv_scores.std()]
})

print(results_comparison)

# 7. Feature Importance (สำหรับ Logistic Regression)
print("\n7. ความสำคัญของตัวแปร (Feature Importance)")
print("-" * 50)

# คำนวณความสำคัญจาก coefficients ของ Logistic Regression
feature_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': np.abs(lr_model.coef_).mean(axis=0)
})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

print("ความสำคัญของตัวแปรตาม Logistic Regression:")
for idx, row in feature_importance.iterrows():
    print(f"{row['Feature']}: {row['Importance']:.4f}")

# 8. การทำนายตัวอย่าง
print("\n8. ตัวอย่างการใช้งานโมเดลทำนาย")
print("-" * 50)

# สร้างข้อมูลผู้ป่วยตัวอย่าง
sample_patients = pd.DataFrame({
    'Age': [45, 65, 70],
    'Blood_Pressure': [120, 150, 180],
    'Albumin_Urine': [0.2, 1.5, 4.0],
    'Random_Blood_Glucose': [100, 180, 250],
    'Serum_Creatinine': [1.0, 2.5, 4.0],
    'Hemoglobin': [14, 11, 9],
    'eGFR': [95, 45, 20],
    'Cholesterol': [180, 220, 280],
    'BMI': [22, 28, 35],
    'Diabetes_Duration': [0, 8, 15]
})

# ทำนายด้วยโมเดลที่ดีที่สุด
sample_scaled = scaler.transform(sample_patients)
best_model = nn_model if nn_accuracy > lr_accuracy else lr_model
predictions = best_model.predict(sample_scaled)
probabilities = best_model.predict_proba(sample_scaled)

print("ผลการทำนายสำหรับผู้ป่วยตัวอย่าง:")
for i, (pred, prob) in enumerate(zip(predictions, probabilities)):
    print(f"\nผู้ป่วยที่ {i+1}:")
    print(f"  - การทำนาย: ระดับ {pred} ({target_labels[pred]})")
    print(f"  - ความน่าจะเป็น: {prob.max():.3f}")

    # แสดงข้อมูลผู้ป่วย
    patient_data = sample_patients.iloc[i]
    print("  - ข้อมูลผู้ป่วย:")
    for col, val in patient_data.items():
        print(f"    {col}: {val:.1f}")

# 9. บทสรุปและข้อเสนอแนะ
print("\n9. บทสรุปและการอภิปรายผล")
print("-" * 50)

print("ข้อดีและข้อเสียของแต่ละเทคนิค:")
print("\nLogistic Regression:")
print("ข้อดี:")
print("- เข้าใจง่าย และตีความผลได้")
print("- เร็วในการฝึกสอนและทำนาย")
print("- ไม่ต้องการข้อมูลจำนวนมาก")
print("- แสดงความสำคัญของตัวแปรได้ชัดเจน")
print("ข้อเสีย:")
print("- จำกัดในการจับความสัมพันธ์ที่ซับซ้อน")
print("- สมมติฐานเชิงเส้นอาจไม่เหมาะกับข้อมูลทางการแพทย์")

print("\nNeural Network:")
print("ข้อดี:")
print("- จับความสัมพันธ์ที่ซับซ้อนได้ดี")
print("- มีความยืดหยุ่นสูงในการเรียนรู้")
print("- ประสิทธิภาพสูงกับข้อมูลจำนวนมาก")
print("ข้อเสีย:")
print("- ต้องการข้อมูลจำนวนมาก")
print("- ใช้เวลาในการฝึกสอนนาน")
print("- ยากต่อการตีความผล (Black box)")

print("\nk-Fold Cross-Validation:")
print("ข้อดี:")
print("- ประเมินประสิทธิภาพแบบครอบคลุม")
print("- ลดปัญหา overfitting")
print("- ใช้ข้อมูลได้อย่างมีประสิทธิภาพ")
print("ข้อเสีย:")
print("- ใช้เวลาในการคำนวณมาก")
print("- อาจไม่เหมาะกับข้อมูลที่มี temporal dependency")

best_model_name = "Neural Network" if nn_accuracy > lr_accuracy else "Logistic Regression"
print(f"\nโมเดลที่ให้ผลลัพธ์ดีที่สุด: {best_model_name}")
print(f"ความแม่นยำ: {max(lr_accuracy, nn_accuracy):.4f}")

print("\nข้อเสนอแนะสำหรับการพัฒนาต่อ:")
print("1. เพิ่มข้อมูลเพื่อปรับปรุงประสิทธิภาพ")
print("2. ทดลองเทคนิคอื่นๆ เช่น Random Forest, SVM")
print("3. ปรับแต่ง hyperparameters")
print("4. เพิ่มการวิเคราะห์ feature engineering")
print("5. ใช้เทคนิค ensemble methods")

print("\n" + "=" * 80)
print("การวิเคราะห์เสร็จสิ้น")
print("=" * 80)

การวิเคราะห์แนวโน้มการเกิดโรคไตเรื้อรัง (CKD)
จากข้อมูลเวชระเบียนอิเล็กทรอนิกส์ด้วยเทคนิคการเรียนรู้ของเครื่อง

1. การโหลดและตรวจสอบข้อมูل
--------------------------------------------------
ข้อมูลพื้นฐาน:
จำนวนตัวอย่าง: 1000
จำนวนฟีเจอร์: 10

ตัวอย่างข้อมูล 5 แถวแรก:
   Age  Blood_Pressure  Albumin_Urine  Random_Blood_Glucose  Serum_Creatinine  \
0   62      157.987109       1.045481            178.656391          1.760706   
1   52      148.492674       0.136579             70.000000          3.276272   
2   64      131.192607       0.857940            132.018516          2.878943   
3   77      117.061264       0.235312            165.055367          2.055260   
4   51      143.964466       3.302971            159.191726          2.629160   

   Hemoglobin        eGFR  Cholesterol        BMI  Diabetes_Duration  Target  
0   12.418940  137.008053   188.483757  30.946132           0.340549       3  
1   10.030932  110.691697   209.801547  28.336226           4.866514       4  
2   11.9